In [22]:
import pandas as pd
train_data=pd.read_excel("/content/u.xlsx")
train_data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [35]:
new = pd.read_excel("/content/u.xlsx")

In [3]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095405 sha256=64e9c4643678fc96b699b6c717d7fcea7afddadd72f8a95056e85e3d3e297a2d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
from surprise import SVD
import numpy as np
import surprise
from surprise import Reader, Dataset
reader = Reader(rating_scale=(1,5))
train_data_mf = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)
trainset = train_data_mf.build_full_trainset()
svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [5]:
train_preds = svd.test(trainset.build_testset())
train_pred_mf = np.array([pred.est for pred in train_preds])

In [6]:
%pylab inline
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg as sla

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['svd']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [7]:
train_sparse_matrix = sparse.csr_matrix((train_data.rating.values, (train_data.userId.values, train_data.movieId.values)))

In [8]:
train_averages = dict()
train_global_average = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
train_averages['global'] = train_global_average

In [9]:
def get_average_ratings(sparse_matrix, of_users):
  ax = 1 if of_users else 0 
  sum_of_ratings = sparse_matrix.sum(axis=ax).A1
  is_rated = sparse_matrix!=0
  no_of_ratings = is_rated.sum(axis=ax).A1
  u,m = sparse_matrix.shape
  average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
  for i in range(u if of_users else m)
  if no_of_ratings[i] !=0}
  return average_ratings

In [10]:
train_averages['user'] = get_average_ratings(train_sparse_matrix, of_users=True)

In [11]:
train_averages['movie'] = get_average_ratings(train_sparse_matrix, of_users=False)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity 

In [18]:
train_averages['user'] = get_average_ratings(train_sparse_matrix, of_users=True)

In [19]:
train_averages['movie'] = get_average_ratings(train_sparse_matrix, of_users=False)

In [48]:
movieID = 242

In [49]:
user_sim = cosine_similarity(train_sparse_matrix[5] , train_sparse_matrix).ravel()
top_sim_users = user_sim.argsort()[::-1][1:] 
top_ratings = train_sparse_matrix[top_sim_users, movieID].toarray().ravel()
top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
top_sim_users_ratings.extend([train_averages['movie'][movieID]]*(5 -len(top_sim_users_ratings)))

In [50]:
movie_sim = cosine_similarity(train_sparse_matrix[:,movieID].T,
train_sparse_matrix.T).ravel()
top_sim_movies = movie_sim.argsort()[::-1][1:]
# we are ignoring 'The User' from its similar users.
# get the ratings of most similar movie rated by this user
top_ratings = train_sparse_matrix[5, top_sim_movies].toarray().ravel()
# we will make it's length "5" by adding user averages to
top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
top_sim_movies_ratings.extend([train_averages['user'][5]]*(5-len(top_sim_movies_ratings)))

In [51]:
import xgboost as xgb
# prepare train data
x_train = new.drop(['userId', 'movieId','rating'], axis=1)
y_train = train_data['rating']
# initialize XGBoost model
xgb_model = xgb.XGBRegressor(silent=False, n_jobs=13,random_state=15,n_estimators=100)
# fit the model
xgb_model.fit(x_train, y_train, eval_metric = 'rmse')

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[08:51:38] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=13, num_parallel_tree=None,
             predictor=None, random_state=15, ...)

In [52]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new.drop(['userId', 'movieId', 'rating'], axis=1),
                                                    train_data['rating'], 
                                                    test_size=0.2, 
                                                    random_state=15)

In [53]:
# dictionaries for storing train and test results
results = dict()

# from the trained model, get the predictions
y_est_pred = xgb_model.predict(X_test)

# get the rmse and mape of test data
rmse_test = np.sqrt(np.mean([ (y_test.values[i] - y_est_pred[i])**2 for i in range(len(y_est_pred)) ]))
mape_test = np.mean(np.abs( (y_test.values- y_est_pred)/y_test.values )) * 100

# store the results in results dictionary
results = {'rmse': rmse_test, 'mape': mape_test, 'predictions': y_est_pred}
print(results)

{'rmse': 0.987092923510167, 'mape': 32.42715809812148, 'predictions': array([3.5758502, 3.373207 , 3.535742 , ..., 4.1403947, 3.8025336,
       3.812086 ], dtype=float32)}
